<a href="https://colab.research.google.com/github/sindhuja279/BISlab/blob/main/CuckooSearch_BIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import random
from scipy.special import gamma  # Import gamma from scipy.special

# Levy flight function
def levy_flight(beta=1.5, d=1):
    sigma_u = np.power((gamma(1 + beta) * np.sin(np.pi * beta / 2) / gamma((1 + beta) / 2) * beta *
                        np.cos(np.pi * beta / 2) ** 2), 1 / beta)
    u = np.random.normal(0, sigma_u, size=d)
    v = np.random.normal(0, 1, size=d)
    step = u / np.power(np.abs(v), 1 / beta)
    return step

# Initialize population (nests)
def initialize_population(n_nests, n_dim, lower_bound, upper_bound):
    return np.random.uniform(lower_bound, upper_bound, (n_nests, n_dim))

# Fitness function (Example: Sphere function)
def fitness_function(x):
    return np.sum(x ** 2)

# Cuckoo Search Algorithm
def cuckoo_search(n_nests, n_dim, lower_bound, upper_bound, max_iter, pa=0.25):
    # Step 1: Initialize nests randomly
    nests = initialize_population(n_nests, n_dim, lower_bound, upper_bound)

    # Step 2: Evaluate fitness of all nests
    fitness = np.array([fitness_function(nest) for nest in nests])

    # Track the best solution found so far
    best_idx = np.argmin(fitness)
    best_nest = nests[best_idx]
    best_fitness = fitness[best_idx]

    # Start iterations
    for iteration in range(max_iter):
        # Generate new solutions using Levy flight
        for i in range(n_nests):
            # Generate a new solution by Levy flight
            step = levy_flight(d=n_dim)
            new_nest = nests[i] + step * (nests[i] - best_nest)

            # Apply boundary conditions
            new_nest = np.clip(new_nest, lower_bound, upper_bound)

            # Evaluate the new solution
            new_fitness = fitness_function(new_nest)

            # If the new solution is better, replace the old nest
            if new_fitness < fitness[i]:
                nests[i] = new_nest
                fitness[i] = new_fitness

                # Update the best solution if necessary
                if new_fitness < best_fitness:
                    best_nest = new_nest
                    best_fitness = new_fitness

        # Abandon some of the worst nests and generate new random solutions
        for i in range(n_nests):
            if random.random() < pa:  # with probability pa
                nests[i] = np.random.uniform(lower_bound, upper_bound, n_dim)
                fitness[i] = fitness_function(nests[i])

        # Print progress every 100 iterations
        if (iteration + 1) % 100 == 0 or iteration == max_iter - 1:
            print(f"Iteration {iteration + 1}, Best Fitness: {best_fitness}")

    return best_nest, best_fitness

# Parameters
n_nests = 25  # Number of nests
n_dim = 10  # Dimensionality of the problem
lower_bound = -5  # Lower bound for the search space
upper_bound = 5  # Upper bound for the search space
max_iter = 1000  # Maximum number of iterations
pa = 0.25  # Probability of abandoning the worst nests

# Run Cuckoo Search
best_solution, best_solution_fitness = cuckoo_search(n_nests, n_dim, lower_bound, upper_bound, max_iter, pa)

print("\nBest solution found: ", best_solution)
print("Best fitness value: ", best_solution_fitness)

Iteration 100, Best Fitness: 16.95161359890179
Iteration 200, Best Fitness: 8.352823050981886
Iteration 300, Best Fitness: 8.352823050981886
Iteration 400, Best Fitness: 8.352823050981886
Iteration 500, Best Fitness: 8.216709341274424
Iteration 600, Best Fitness: 5.472407510615651
Iteration 700, Best Fitness: 3.9986633949411554
Iteration 800, Best Fitness: 3.9986633949411554
Iteration 900, Best Fitness: 3.9986633949411554
Iteration 1000, Best Fitness: 3.259505337316245

Best solution found:  [ 0.58093095  0.46384379  0.58922097 -0.12935989  0.38801827  0.52285076
 -0.26379424 -0.8573545  -0.19800433  1.03690753]
Best fitness value:  3.259505337316245
